In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Import the requests library.
import requests
# Import the API key.
from config import weather_api_key
from config import g_key
import sys
from datetime import datetime
import traceback
import gmaps

In [2]:
#Helper function to convert a yes input string to a True boolean value
def yesno_str_to_bool(user_str):
    return (user_str == "yes")

In [3]:
#Read the CSV into a dataframe
weather_df = pd.read_csv("analysis/WeatherPy_challenge.csv") 

In [4]:
#User Inputs
min_temp=float(input("What min temperature are you interested in?"))
max_temp=float(input("What max temperature are you interested in?"))
rain=input("Do you want it to be raining? (yes/no) ")
snow=input("Do you want it to be snowing? (yes/no) ")

What min temperature are you interested in?40
What max temperature are you interested in?80
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [5]:
#Get initial boolean values
include_rain = yesno_str_to_bool(rain)
include_snow = yesno_str_to_bool(snow)

In [ ]:
#Filter out the data between the min-max temperature ranges from the user input
valid_temp_range=weather_df[(weather_df["Max Temp"] >= min_temp) & (weather_df["Max Temp"] <= max_temp)]
valid_temp_range.head()

In [7]:
#Filter based on rain in the last 3 hours
if include_rain:
    valid_data_rain = valid_temp_range[valid_temp_range["Rain inches (last 3 hrs)"]>0]
else:
    valid_data_rain = valid_temp_range[valid_temp_range["Rain inches (last 3 hrs)"]==0]

In [8]:
valid_data_rain.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,vaini,to,"04/19/2020, 11:03:09",-21.20,-175.20,77.00,83,20,9.17,few clouds,0.0,0.0
3,vaini,to,"04/19/2020, 11:03:10",-21.20,-175.20,77.00,83,20,9.17,few clouds,0.0,0.0
4,busselton,au,"04/19/2020, 11:03:10",-33.65,115.33,55.99,58,97,14.70,overcast clouds,0.0,0.0
5,hillsborough,gd,"04/19/2020, 11:03:10",37.57,-122.38,61.00,62,75,5.44,broken clouds,0.0,0.0
8,ushuaia,ar,"04/19/2020, 11:01:01",-54.80,-68.30,44.60,75,75,19.46,drizzle,0.0,0.0


In [9]:
#Additional Filter based on snow in the last 3 hours
if include_snow:
    valid_data = valid_data_rain[valid_data_rain["Snow inches (last 3 hrs)"]>0]
else:
    valid_data = valid_data_rain[valid_data_rain["Snow inches (last 3 hrs)"]==0]

In [10]:
valid_data.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,vaini,to,"04/19/2020, 11:03:09",-21.20,-175.20,77.00,83,20,9.17,few clouds,0.0,0.0
3,vaini,to,"04/19/2020, 11:03:10",-21.20,-175.20,77.00,83,20,9.17,few clouds,0.0,0.0
4,busselton,au,"04/19/2020, 11:03:10",-33.65,115.33,55.99,58,97,14.70,overcast clouds,0.0,0.0
5,hillsborough,gd,"04/19/2020, 11:03:10",37.57,-122.38,61.00,62,75,5.44,broken clouds,0.0,0.0
8,ushuaia,ar,"04/19/2020, 11:01:01",-54.80,-68.30,44.60,75,75,19.46,drizzle,0.0,0.0


In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = valid_data[["City", "Country", "Max Temp", "Current Description","Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,vaini,to,77.00,few clouds,-21.20,-175.20,
3,vaini,to,77.00,few clouds,-21.20,-175.20,
4,busselton,au,55.99,overcast clouds,-33.65,115.33,
5,hillsborough,gd,61.00,broken clouds,37.57,-122.38,
8,ushuaia,ar,44.60,drizzle,-54.80,-68.30,
10,santa luzia,br,73.40,broken clouds,-19.77,-43.85,
12,vila velha,br,79.00,few clouds,-20.33,-40.29,
13,hermanus,za,63.00,broken clouds,-34.42,19.23,
15,kapaa,us,71.01,scattered clouds,22.08,-159.32,
18,abasolo,mx,77.00,scattered clouds,20.45,-101.52,


In [12]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [ ]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    lat_long = f"{lat},{lng}"
    params["location"] = lat_long
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    if len(hotels["results"]) == 0:
        print(f"Hotel results == 0... skipping. {lat_long}")
        continue
    try:
        hotel_result = hotels["results"]
        hotel_df.loc[index, "Hotel Name"] = hotel_result[0]["name"]
    except (IndexError):
        print(f"Hotel not found... skipping. {hotel_result}")
    except:
        print(f"Unexpected error: {sys.exc_info()[0]}")
        print(f"Unexpected error[1]: {sys.exc_info()[1]}")

Hotel results == 0... skipping. 42.6,-73.97
Hotel results == 0... skipping. 42.6,-73.97
Hotel results == 0... skipping. 42.6,-73.97
Hotel results == 0... skipping. 42.6,-73.97
Hotel results == 0... skipping. 42.6,-73.97
Hotel results == 0... skipping. 42.6,-73.97
Hotel results == 0... skipping. 33.19,-82.53
Hotel results == 0... skipping. 42.6,-73.97
Hotel results == 0... skipping. 45.38,24.4
Hotel results == 0... skipping. 42.6,-73.97
Hotel results == 0... skipping. 42.6,-73.97
Hotel results == 0... skipping. 42.6,-73.97
Hotel results == 0... skipping. 42.6,-73.97
Hotel results == 0... skipping. 42.6,-73.97
Hotel results == 0... skipping. 38.5,48.67
Hotel results == 0... skipping. 33.19,-82.53
Hotel results == 0... skipping. 42.6,-73.97
Hotel results == 0... skipping. 42.6,-73.97
Hotel results == 0... skipping. 40.28,97.2
Hotel results == 0... skipping. 42.6,-73.97
Hotel results == 0... skipping. 42.6,-73.97
Hotel results == 0... skipping. 51.03,90.23
Hotel results == 0... skipping. -

In [ ]:
#Set HTML template for the popup on each marker on GMaps
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Descritpion</dt><dd>Description: {Current Description} with a max temp: {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [ ]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#Ensure there is at least 1 location and apply the info boxes to each pin.
if len(locations) > 0:
    # Get the maximum temperature.
    max_temp = hotel_df["Max Temp"]
    #GMaps weights doesn't allow negative values.
    temps = []
    for temp in max_temp:
        temps.append(max(temp, 0))

    #heat_layer = gmaps.heatmap_layer(locations,weights=temps,dissipating=False,max_intensity=300, point_radius=4)
    heat_layer = gmaps.heatmap_layer(locations,weights=temps)
    marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
    fig.add_layer(heat_layer)
    fig.add_layer(marker_layer)
    #Save hotel dataframe to a .CSV
    hotel_df.to_csv('./analysis/WeatherPy_vacation.csv')

# Call the figure to plot the data.
fig

In [ ]:
hotel_df.head(15)